In [135]:
from numpy import cos, sin, radians, pi, sqrt, interp, log10, log, asarray, abs
import pandas as pd

Informações enunciado

In [136]:
m = 4                  #modulo
Np, Ng= 18, 54         #número de dentes 
theta = 20             #angulo de pressão
ωp = 1500              #rotação pinhão
sy = 330               #MPa
n = 3.5                #coeficiente de segurança
η = 8                  #eficiencia  #%
v = 0.29               #coeficiente de poisson

In [137]:
Ep = Eg = 186*10**3     #MPa
vp = vg = v 

cte = [2.47*10**(-1),
      0.657*10**(-3),
     -1.186*10**(-7)]

Av = 10                 #assumindo indice de qualidade 10

In [138]:
ZW = 1    #fator de razão de dureza para resistencia ao crateramento    #O valor de ZW para o pinhão é fixado em 1,0.
Y0 = 1    #fator de temperatura     #1 p/ até 120C

In [139]:
Lp = Lg = 55                #meses
qp = qg = 1                 #p/ rotação em 1 direção só
confp = confg = 90          #confiabilidade #%
ft = [3.12, 1.69]           #tabelado em função do número de dentes

In [140]:
r = [ft[0]*m, ft[1]*m] 

In [141]:
def de(N):
    return m*( N + 2 )                        #d_externo

def dr(N):
    return m*( N - 2.5 )                      #d_raiz

def dp(N):
    return m*N                                #d_primitivo  
         
def db(N):
    return cos(radians(theta))*dp(N)          #d_base

def V(N, ω):
    return (pi*ω/30)*(dp(N)/2000)             #velocidade linear

def KV(N, ω):
    return (6.1 + V(N, ω))/6.1                #fator de velocidade

In [142]:
p = m*pi                                 #passo circular
e = p/2                                  #espessura do dente
r = 0.3*m                                #raio do filete 
i = Ng/Np                                #relação de engrenamento
b = 12*m                                 #largura da face 6 a 16*m  
f = 0.3*m                                #cordão raiz do dente (ou raio do filete)
d_centros = ( dp(Np) + dp(Ng) )/2        #Distancia entre centros 

In [143]:
##FATOR DE FORMA##
def Y(N):
    N_l = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 28, 30, 34, 38, 43, 50, 60, 75, 100, 150, 300, 400]
    Y_l = [0.2450, 0.2610, 0.2770, 0.2900, 0.2960, 0.3030, 0.3090, 0.3140, 0.3220, 0.3280, 0.3310, 0.3370, 0.3460, 0.3530, 0.3590, 0.3710, 0.3840, 0.3970, 0.4090, 0.4220, 0.4350, 0.4470, 0.4600, 0.4720, 0.4800]
    return interp(N, N_l, Y_l)

In [144]:
σmax = sy/n                                                                                  #tensão máxima
ωg = ωp/i                                                                                    #rotação na coroa

def wt(N, ω):
    return ( sy*b*m*Y(N) ) / ( n*KV(N, ω) )                                                  #carga tangecial

def T(N, ω):                                                                                 #torque
    return wt(N, ω)*(dp(N)/2000)

Pp, Pg = wt(N=Np, ω=ωp)*V(N=Np, ω=ωp), ( T(N=Ng, ω=ωg) * ωg )*(100 - η)/ ( 100 * 9.55 )      #potencia


In [145]:
## CONSTANTES PARA CALCULAR O FATOR DINAMICO kv ##
B = 0.25 * (Av - 5)**(2/3)
C = 3.5637 + 3.9914*( 1 - B )

In [146]:
Khma = cte[0] + cte[1]*b + cte[2]*(b**2)         #fator de alinhamento de engrenamento
Khe = 1                                          #fator de ajuste
Khpm = 1                                         #fator de carga de deflexão
Khmc = 1                                         #fator de formato da face do dente

def Khpf(N):
    return ( b/(10*dp(N)) ) - 0.0375 + 0.000492*b   #fator de proporção do pinhão

In [162]:
def Ko():
    return 1.25                                          #fator de sobrecarga      #motor elétrico que aciona uma bomba centrífuga de baixa velocidade

def Kh(N):
    return 1 + Khmc * ( Khpf(N) * Khpm + Khma * Khe)    #fator de distribuição de carga

def kv(N, ω):
    return ( C / (C + sqrt(V(N, ω)) ) )**(-B)         #fator dinâmico

#def Ks():
#    return 1                                             #fator de tamanho

if b>=2.79 and b<=51:
    def Ks(N):
        return 1/( 1.18337*(b*m*sqrt(Y(N)))**(-0.0535) )   #p/ b de 2,79 a 51mm

if b>=52 and b<=254:
    def Ks(N):
        return 1/( 1.18337*(b*m*sqrt(Y(N)))**(-0.0535) )   #p/ b de 52 a 254mm


In [163]:
def Zr():
    return 1                                                                   #fator de condição de superfície

def Ze():
    return sqrt( 1/( pi * ( ( (1 - vp**2) / Ep) + (( 1 - vg**2 ) / Eg) ) ) )   #coeficiente de elasticidade

In [164]:
def C_c():
    return cos(radians(theta)) * sin(radians(theta)) * i / ( 2*(i+1) )   #fator de curvatura na linha primitiva

def C_1(N):
     return N*sin(radians(theta))/2

def C_2(N):
    return C_1(N)*i

def C_3(N):
    return pi*cos(radians(theta))

def C_4(N):
     return 0.5 * ( sqrt( ((N + 2)**2) - (N*cos( radians(theta)))**2 ) - sqrt( (N**2) - (N*cos( radians(theta) ))**2 ) )

def C_x(N):
    return (C_1(N) - C_3(N) + C_4(N))*(C_2(N) + C_3(N) - C_4(N)) / ( C_1(N)*C_2(N) )  # fator para ajuste da altura específica do LPSTC (ponto extremo inferior de contato de um dente)

In [165]:
## FATOR GEOMÉTRICO PARA RESISTÊNCIA AO CRATERAMENTO ##

def Zi(N):
    return C_c()*C_x(N)                

In [166]:
## TENSÃO DE CONTATO ##

def 𝜎H(N, ω):
    return 𝑍e()*sqrt( ( wt(N, ω) * 𝐾o() *  kv(N, ω) * 𝐾s(N) * 𝐾h(N) * 𝑍r() ) / (dp(N) * 𝑏 * 𝑍i(N)) )  

In [168]:
## FATOR DE CONFIABILIDADE ##
## CONSIDERANDO A TABELA ##

def YZ(conf):
    data_yz = {
        'confiabilidade': [99.99, 99.9, 99, 90, 50],
        'Yz': [1.5, 1.25, 1, 0.85, 0.70]
    }
    df_yz = pd.DataFrame(data_yz)
    return float(df_yz.loc[df_yz['confiabilidade']==float(conf)]['Yz'])

## CONSIDERANDO A EQUAÇÃO ##

#def YZ(conf):
#    return 0.658 - 0.0759*log(1-conf/100)   #0,5<=conf<=0,99
#
#def YZ(conf):
#    return 0.5 - 0.109*log(1-conf/100)      #0,99<=conf<=0,9999

In [154]:
def nL(L, ω, q):
    return 60*L*ω*q*720         #número de ciclos de tensão

def ZN(L, ω, q):
    return 1.4488*nL(L, ω, q)**(-0.023)    #fator do ciclagem de tensão para resistência ao crateramento
  
def HB(N, ω, L, q, conf):
    return ( ( 𝜎H(N, ω)*Y0*YZ(conf) / ( ZN(L, ω, q)*ZW ) ) - 200 ) / 2.22      #dureza brinel preliminar #considerando SH = 1

In [169]:
def brinell(value):
    data_brinell = [111,116,121,126,131,137,143,149,156,163,167,170,174,179,183,187,192,197,201,207,212,217,223,229,235,
     241,248,255,262,269,277,285,293,302,311,321,331,341,352,363,375,388,401,415,429,444,461,477,495,514,534,555,578,601,
     627,653,682,712,745]

    def find_nearest(array, value):
        array = asarray(array)
        idx = (abs(array - value)).argmin() 
        return array[idx]

    if find_nearest(data_brinell, value)>=value:
        idx = find_nearest(data_brinell, value)
    else: 
        array = asarray(data_brinell)
        idx = array[(abs(array - value)).argmin()  + 1]

    return idx       #dureza brinel (entra o valor preliminar HB e retorna o valor mais próximo (acima) dos valores tabelados de dureza)

In [170]:
def 𝜎HB(N, ω, L, q, conf):
    return brinell(HB(N, ω, L, q, conf))*2.22 + 200          #Número ao contato permitido

def SH(N, ω, L, q, conf):
    return 𝜎HB(N, ω, L, q, conf)*ZN(L, ω, q)*ZW/(𝜎H(N, ω)*Y0*YZ(conf))    #fator de segurança AGMA ao contato

In [187]:
data = {
'': ['Pinhão', 'Coroa'],
'Diâmetro externo [mm]:': [round(de(Np), 2) , round(de(Ng), 2)],
'Diâmetro raiz [mm]:': [round(dr(Np), 2) , round(dr(Ng), 2)],
'Diâmetro primitivo [mm]:': [round(dp(Np), 2) , round(dp(Ng), 2)],
'Diâmetro base [mm]:': [round(db(Np), 2) , round(db(Ng), 2)],
'Cordão raiz do dente [mm]:': [round(f, 2) , round(f, 2)] ,
'Velocidade linear em [m/s]:': [round(V(N=Np, ω=ωp), 2) , round(V(N=Ng, ω=ωg), 2)] ,
'Largura da face nominal [mm]:': [round(b, 2) , round(b, 2)],
'Relação de engrenamento (i):': [round(i, 2) , '-'],
'Distância entre centros das engrenagens [mm]:': [round(d_centros, 2) , '-'],
'Passo circular:': [round(p, 2) , '-'],
'Espessura do dente [mm]:': [round(e, 2) , round(e, 2)],
'Tensão máxima admissível [MPa]:': [round(σmax, 2) , round(σmax, 2)],
'Fator de velocidade:': [round(KV(N=Np, ω=ωp), 3) , round(KV(N=Np, ω=ωp), 3)],
'Fator de forma:': [round(Y(Np), 3) , round(Y(Ng), 3)],
'Carga tangencial [N]:': [round(wt(N=Np, ω=ωp), 2) , round(wt(N=Ng, ω=ωg), 2)],
'Potencia [W]:': [round(Pp, 2) , round(Pg, 2)],
'Torque [N*m]:': [ round( T( N=Np, ω=ωp ), 2 ) , round( T( N=Ng, ω=ωg ), 2 ) ],
'Rotação [rpm]: ': [round(ωp ,2) , round(ωg, 2)],
'Tensão de contato [MPa]': [ round(𝜎H(N=Np, ω=ωp), 2), round(𝜎H(N=Ng, ω=ωg), 2) ],
'Coeficiente de elasticidade (Ze) [MPa^(1/2)]': [round(Ze(), 2), round(Ze(), 2)],
'Fator de sobrecarga (Ko)': [round(Ko(), 2), round(Ko(), 2)],
'Fator dinâmico (Kv)': [round( kv(N=Np, ω=ωp), 2 ), round( kv(N=Ng, ω=ωg), 2 ) ],
'Fator de tamanho (Ks)': [ round(Ks(N=Np), 2), round(Ks(N=Ng), 2)],
'Distribuição de carga (Kh)': [ round(Kh(dp(Np)), 2), round(Kh(dp(Ng)), 2)],
'Fator geométrico (Zi)': [round(Zi(Np), 2), round(Zi(Ng), 2)],
'Fator de ciclagem de tensão para resistência ao crateramento (Zn)' : [ round(ZN(L=Lp, ω=ωp, q = qp), 2), '-'],
'Dureza Brinell': [ round(brinell(HB(N=Np, ω=ωp, L=Lp, q=qp, conf=confp)), 2), '-' ],
'Número ao contato permitido AGMA (𝜎HB) [MPa]': [ round(𝜎HB(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2),'-' ],
'Fator de segurança AGMA [MPa]': [round(SH(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2), '-']
}

In [188]:
df = pd.DataFrame(data)

In [189]:
display(df)

,,Diâmetro externo [mm]:,Diâmetro raiz [mm]:,Diâmetro primitivo [mm]:,Diâmetro base [mm]:,Cordão raiz do dente [mm]:,Velocidade linear em [m/s]:,Largura da face nominal [mm]:,Relação de engrenamento (i):,Distância entre centros das engrenagens [mm]:,...,Coeficiente de elasticidade (Ze) [MPa^(1/2)],Fator de sobrecarga (Ko),Fator dinâmico (Kv),Fator de tamanho (Ks),Distribuição de carga (Kh),Fator geométrico (Zi),Fator de ciclagem de tensão para resistência ao crateramento (Zn),Dureza Brinell,Número ao contato permitido AGMA (𝜎HB) [MPa],Fator de segurança AGMA [MPa]
0,Pinhão,80,62.0,72,67.66,1.2,5.65,48,3.0,144.0,...,179.78,1.25,1.35,1.08,1.28,0.10,0.87,269,797.18,1.01
1,Coroa,224,206.0,216,202.97,1.2,5.65,48,-,-,...,179.78,1.25,1.35,1.09,1.27,0.12,-,-,-,-


In [161]:
#df.to_excel('calculos.xlsx', index=False)